# Third Project


In this third project, you will do some analysis and optimizations in uCIR. First, you will determine the basic blocks. After, you're going to build a control flow graph (CFG) adding additional predecessor and successor information to nodes of the CFG.  Next, you're going to compute the dominators for the CFG, i.e. for each basic block, identify the set of basic blocks that dominate that block. This means that every path from the entry block to that block must go through the blocks in the dominator set. After that, you can simplify control flow (1) by merging consecutive blocks where the parent has one child, the child one parent, and both have compatible instruction leaders; (2) find all immediate dead blocks and remove then and finally, perform some basic optimizations like constant propagation, constant folding and dead code elimination.

We will try to give you a hand by giving you some of the pieces prewritten and littered your code with helpful statements. By the time you’re done, you’ll have a pretty thorough understanding of the analyses and optimizations realized by real compilers.

## Basic Block Creation

After constructing the sequence of instructions in its intermediate representation for the program, we will determine the basic blocks. A basic block is a sequence of instructions where the control flow enters only at the beginning of the block and exits at the end of the block, without the possibility of deviation to any other part of the program. The basic blocks make up the nodes of a control flow graph (CFG), a structure that will be important for performing code optimizations.

To create the basic blocks, you must first determine the set of leaders, which will match the first statement of each basic block. To do this, proceed according to the algorithm below:

1. The first instruction is a leader;
2. Any instruction that is subject to conditional or unconditional deviation is a leader;
3. Any instruction that comes immediately after a conditional or unconditional deviation instruction is a leader;
4. For each leader, your basic block consists of the leader and all instructions that follow him to the next leader, excluding this last.

When building the basic blocks, you may note that the first statement of a basic block is always a label and the last statement is a jump statement (conditional or unconditional). Function calls should be treated as straight-line IR nodes (i.e., they are not treated as branches; their successor is the instruction immediately after the call). Return nodes do not have any successors.

  - Unconditional jumps (like at the end of for loops) have only one successor:
the target of the jump statement. When you see an unconditional jump, add the target of the jump statement as a successor of the jump, and the jump statement as a predecessor of the target.

  - Conditional jumps have two successors: the ```fall_through``` target, which can be a successor in the linked list, and the ```taken``` target. Add the branch as a predecessor of the taken target, and the taken target as an additional successor of the branch.

### Sample Code

The sample code below defines classes and functions for creating and navigating
basic blocks.  You need to write all of the code needed yourself.

In [4]:
# An example of how to create basic blocks

class Block(object):
    def __init__(self, label):
        self.label = label       # Label that identifies the block
        self.instructions = []   # Instructions in the block
        self.predecessors = []   # List of predecessors
        self.next_block = None   # Link to the next block

    def append(self,instr):
        self.instructions.append(instr)

    def __iter__(self):
        return iter(self.instructions)

class BasicBlock(Block):
    '''
    Class for a simple basic block.  Control flow unconditionally
    flows to the next block.
    '''
    def __init__(self, label):
        super(BasicBlock, self).__init__(label)
        self.branch = None  # Not necessary the same as next_block in the linked list

class ConditionBlock(Block):
    """
    Class for a block representing an conditional statement.
    There are two branches to handle each possibility.
    """
    def __init__(self, label):
        super(ConditionBlock, self).__init__(label)
        self.taken = None
        self.fall_through = None
        
class BlockVisitor(object):
    '''
    Class for visiting basic blocks.  Define a subclass and define
    methods such as visit_BasicBlock or visit_IfBlock to implement
    custom processing (similar to ASTs).
    '''
    def visit(self,block):
        while isinstance(block,Block):
            name = "visit_%s" % type(block).__name__
            if hasattr(self,name):
                getattr(self,name)(block)
            block = block.next_block

## Viewing the Control Flow Graph
Let's create a graphic object and assemble the graphic by adding the generated CFG nodes and edges. For this we will use graphviz (```pip3 install graphviz```), a package that facilitates the creation and rendering of graph descriptions in the DOT language of Graphviz graphical drawing software.

In the code below, we use the option/viewing method to directly inspect the resulting file in PDF format.
The graphics can also be saved in files containing the generated DOT source code.

In [ ]:
def format_instruction(t):
    # Auxiliary method to pretty print the instructions 
    op = t[0]
    if len(t) > 1:
        if op == "define":
            return f"\n{op} {t[1]}"
        else:
            _str = "" if op.startswith('global') else "  "
            if op == 'jump':
                _str += f"{op} label {t[1]}"
            elif op == 'cbranch':
                _str += f"{op} {t[1]} label {t[2]} label {t[3]}"
            elif op == 'global_string':
                _str += f"{op} {t[1]} \'{t[2]}\'"
            elif op.startswith('return'):
                _str += f"{op} {t[1]}"
            else:
                for _el in t:
                    _str += f"{_el} "
            return _str
    elif op == 'print_void' or op == 'return_void':
        return f"  {op}"
    else:
        return f"{op}"

class CFG(object):

    def __init__(self, fname):
        self.fname = fname
        self.g = Digraph('g', filename=fname + '.gv', node_attr={'shape': 'record'})

    def visit_BasicBlock(self, block):
        # Get the label as node name
        _name = block.label
        if _name:
            # get the formatted instructions as node label
            _label = "{" + _name + ":\l\t"
            for _inst in block.instructions[1:]:
                _label += format_instruction(_inst) + "\l\t"
            _label += "}"
            self.g.node(_name, label=_label)
            if block.branch:
                self.g.edge(_name, block.branch.label)
        else:
            # Function definition. An empty block that connect to the Entry Block
            self.g.node(self.fname, label=None, _attributes={'shape': 'ellipse'})
            self.g.edge(self.fname, block.next_block.label)

    def visit_ConditionBlock(self, block):
        # Get the label as node name
        _name = block.label
        # get the formatted instructions as node label
        _label = "{" + _name + ":\l\t"
        for _inst in block.instructions[1:]:
            _label += format_instruction(_inst) + "\l\t"
        _label +="|{<f0>T|<f1>F}}"
        self.g.node(_name, label=_label)
        self.g.edge(_name + ":f0", block.taken.label)
        self.g.edge(_name + ":f1", block.fall_through.label)

    def view(self, block):
        while isinstance(block, Block):
            name = "visit_%s" % type(block).__name__
            if hasattr(self, name):
                getattr(self, name)(block)
            block = block.next_block
        # You can use the next stmt to see the dot file
        # print(self.g.source)
        self.g.view()

#        
# At the end of visit_Program method in the Code Generator Class you call CFG view method
#

    def visit_Program(self):
        # ...
        if self.viewcfg:  # evaluate to True if -cfg flag is present in command line
            for _decl in node.gdecls:
                if isinstance(_decl, FuncDef):
                    dot = CFG(_decl.decl.name.name)
                    dot.view(_decl.cfg)  # _decl.cfg contains the CFG for the function

### Example:
```
int checkPrime(int n) {
    int i, isPrime = 1;
    for (i = 2; i <= n/2; ++i) {
        if (n % i == 0) {
            isPrime = 0;
            break;
        }
    }
    return isPrime;
}
```

<img src="checkPrime.gv.png" alt="Drawing" style="width: 400px;"/>

## DataFlow Analysis

### Reaching Definitions
Reaching definitions is one of the most common and useful dataflow analysis. By knowing where in a program each variable ```x``` may have been defined when control reaches each point ```p```, we can determine many things about ```x```. For just one example, a compiler then knows whether ```x``` is a ```constant``` at point ```p```.

We say a definition ```d``` reaches a point ```p``` if there is a path from the point immediately following ```d``` to ```p```, such that ```d``` is not "killed" along that path. We kill a definition of a variable ```x``` if there is any other definition of ```x``` anywhere along the path. Intuitively, if a definition ```d``` of some variable ```x``` reaches point ```p```, then ```d``` might be the place at which the value of ```x``` used at ```p``` was last defined.

A definition of a variable ```x``` is a statement that assigns a value to ```x``` (For the sake of simplicity, assume that we are dealing only with variables that have no aliases.).

**Iterative Algorithm for Reaching Definitions**

For a generic instruction, we define the GEN and KILL sets as follows:

  - GEN [$d:y \leftarrow f(x_{1},\cdots ,x_{n})]=\{d\}$, a set of locally available definitions in a basic block
  
  - KILL [$d:y \leftarrow f(x_{1},\cdots ,x_{n})]=$ DEFS$[y]-\{d\}$, a set of definitions (not locally available, but in the rest of the program) killed by definitions in the basic block.
  
where DEFS[y] is the set of all definitions that assign to the variable y. Here d is a unique label attached to the assigning instruction; thus, the domain of values in reaching definitions are these instruction labels.

We assume that every control-flow graph has an ENTRY node, which represents the starting point of the graph, and an EXIT node to which all exits out of the graph go. Since no definitions reach the beginning of the graph, the transfer function for the ENTRY block is a simple constant function that returns $\phi$ as an answer. 

The reaching definitions problem is defined by the following equations:

OUT[ENTRY] = $\phi$

and for all basic blocks B other than ENTRY,

$OUT[B] = gen_{B} \bigcup\ (IN[B] - kill_{B})$

$IN[B] = \bigcup_{p\ a\ predecessor\ of\ B} OUT[P]$

These equations can be solved using the following algorithm:
```
// Initialize
for all CFG block b in B,  
  OUT[b] = emptyset; 

// put all blocks into the changed set
// B is all blocks in graph, 
Changed = B; 

//Iterate 
while (Changed != emptyset)
{
  choose a block b in Changed;
  // remove it from the changed set
  Changed = Changed - { b }; 

  // init IN[b] to be empty
  IN[b] = emptyset;  

  // calculate IN[b] from predecessors' OUT[p]
  for all blocks p in predecessors(b) 
     IN[b] = IN[b] Union OUT[p]; 

  oldout = OUT[b]; // save old OUT[b]
  
  // update OUT[b] using transfer function f_b ()
  OUT[b] = GEN[b] Union (IN[b] - KILL[b]);   

  // any change to OUT[b] compared to previous value?
  if (OUT[b] changed) // compare oldout vs. OUT[b]
  {  
    // if yes, put all successors of b into the changed set
    for all blocks s in successors(b) 
       Changed = Changed U { s };  
  }
}
```

### Liveness Analysis
Liveness analysis is a data flow analysis that finds what variables are live after any statement. It is an any-path, backward flow analysis. We will perform intra-procedural liveness analysis (i.e., we will compute liveness at the function granularity, but not across functions).

Once you’ve built the CFG, liveness analysis is easy. Compute the GEN and KILL sets for each block. GEN represents all the temporaries and variables that are used in an instruction, and KILL represents all the temporaries and variables that are defined in an instruction. For most instructions, this should be pretty straightforward. A few tricky cases:

  - ```store_type``` this instruction use source (GEN) & define target (KILL) while ```store_type_*```use both source & target. The same for ```read_type```and ```read_type_*```. Note that this strategy means that you do not touch the ```alloc_type``` instructions. An extra pass can be made at final to remove unused allocs.
  - ```call``` instructions require special care. Because we do not analyze liveness across functions, we must make conservative assumptions about what happens in function calls. In particular, we GEN all the temporaries in ```param_type```. The ```call``` itself kills the return var. The GEN set for any ```call``` instruction therefore contains all global variables, while the KILL set is their return var.

Once you know the GEN and KILL sets for each CFG node, you can compute liveness. To do this, define IN (live-in) and OUT (live-out) sets for each CFG Node. Initialize the OUT sets for Exit nodes to all global variables (because global variables may be used after the function returns), and initialize all other sets to empty. Then compute the live-in and live-out sets for node as follows:

  - The set of variables that are live out of a node is the union of all the variables that are live in to the node's successors.
  - The set of variables that are live in to a node is the set of variables that are live out for the node, minus any variables that are killed by the node, plus any variables that are gen-ed by the node.
  
Note that in these definitions are recursive: the live-out set of a node is defined in terms of the live-in sets of its successors, which are in turn defined in terms of the live-in sets of their successors, and so on. If there is a loop in the code, then the definition seems circular.

The trick to computing liveness is to compute a fixpoint: assignments to each of the live-in and live-out sets so that if you try to compute any node's live-in or live-out set again, you'll get the same result you already have. To do this, we will use a worklist algorithm:

  1. Put all of the IR nodes on the worklist
  2. Pull an IR node off the worklist, and compute its live-out and live-in sets according to the definitions above.
  3. If the live-in set of the node gets updated by the previous step, put all of the node's predecessors on the worklist (because they may need to update their live-out sets).
  4. Repeat steps 2 and 3 until the worklist is empty. The live-out sets of each node now represent a fixpoint.

## Constant Propagation
Constants assigned to a variable can be propagated through the flow graph and substituted at the use of the variable.

Example:
In the code fragment below, the value of x can be propagated to the use of x.
```
int x = 14;
int y = 7 - x / 2;
return y * (28 / x + 2);
```  
Below is the code fragment after ```constant propagation``` and ```constant folding``` (which would likely be further optimized by ```dead code elimination``` of both x and y).
```
int x = 14;
int y = 0;
return 0;
```  
Constant propagation is implemented in compilers using ```reaching definition``` analysis results. If all variable's reaching definitions are the same assignment which assigns a same constant to the variable, then the variable has a constant value and can be replaced with the constant.

Constant propagation can also cause conditional branches to simplify to one or more unconditional statements, when the conditional expression can be evaluated to true or false at compile time to determine the only possible outcome.

## Dead Code Elimination
Code that is unreachable or that does not affect the program (e.g. dead stores) can be eliminated.

Example:
In the example below, the value assigned to i is never used, and the dead store can be eliminated. The first assignment to global is dead, and the third assignment to global is unreachable; both can be eliminated.

```
int global;
void f ()
{
  int i;
  i = 1;          /* dead store */
  global = 1;     /* dead store */
  global = 2;
  return;
  global = 3;     /* unreachable */
}
```  
Below is the code fragment after dead code elimination.
```
int global;
void f ()
{
  global = 2;
  return;
}
```

## Examples
```
int main() {
    int i = 3, n = 6;
    for (int k = 1; k < n; k++) {
        if (i >= n) {
            break;
        }
        else {
            i++;
        }
    }
    assert i == n;
    return 0;
}
```

<img src="opt_ex1.png" alt="Drawing" style="width: 900px;"/>

```
int main () {
    int x, y;
    x = 1;
    y = 1;
    print(x);
    if (y > 1) {
        x = 2;
    }
    print(x);
    return 0;
}
```

<img src="opt_ex2.png" alt="Drawing" style="width: 500px;"/>

## A note on the evaluation

We will award points based on the performance of the code in this step. Unlike the previous step, in which we evaluate the correctness of the execution, in this step, we will award points based on the least amount of instructions from the uCIR. In other words, teams that do a better job of optimizing will receive more points.